In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pulp
import sys
import os
import importlib.util
import importlib
import gurobipy as gp

# モジュールを強制的に再ロード（古いキャッシュをクリア）
if 'code' in sys.modules:
    del sys.modules['code']
# サブモジュールも削除
for module_name in list(sys.modules.keys()):
    if module_name.startswith('code.'):
        del sys.modules[module_name]

current_dir = os.getcwd()
spec = importlib.util.spec_from_file_location("code", os.path.join(current_dir, "code", "__init__.py"))
code_module = importlib.util.module_from_spec(spec)
sys.modules["code"] = code_module
spec.loader.exec_module(code_module)

# サブモジュールも再ロード
importlib.reload(code_module)

# primal_multi_agentから直接インポート
from code.primal_multi_agent import (
    solve_mechanism_2agents,
    solve_mechanism_2agents_iterative,
    save_results_2agents,
    load_results_2agents
)

# utilsからグリッド生成関数をインポート
from code.utils import make_tensor_grid_2d

print("モジュールを再ロードしました")


In [ ]:
home_license = os.path.expanduser('~/gurobi.lic')
os.environ['GRB_LICENSE_FILE'] = home_license

SOLVER = pulp.GUROBI(msg=True)


# ケース1: 2人2財


In [ ]:
# ケース1: 両エージェントとも財a, 財bはBeta(1,1)
NX1, NY1 = 10, 10  # エージェント1のグリッドサイズ
NX2, NY2 = 10, 10  # エージェント2のグリッドサイズ
BETA_PARAMS_1 = [
    (1.0, 1.0),  # 財a
    (1.0, 1.0),  # 財b
]
BETA_PARAMS_2 = [
    (1.0, 1.0),  # 財a
    (1.0, 1.0),  # 財b
]

points1, weights1 = make_tensor_grid_2d(NX1, NY1, BETA_PARAMS_1)
points2, weights2 = make_tensor_grid_2d(NX2, NY2, BETA_PARAMS_2)
print(f"#types agent1 = {len(points1)}")
print(f"#types agent2 = {len(points2)}")
print(f"Total combinations = {len(points1) * len(points2)}")


In [ ]:
status1, obj_val1, u1_sol, u2_sol, p1_sol, p2_sol, n_iter1 = solve_mechanism_2agents_iterative(
    points1, weights1, (NX1, NY1),
    points2, weights2, (NX2, NY2),
    solver=SOLVER
)

print("LP status:", status1)
print("Optimal revenue:", obj_val1)
print(f"Number of iterations: {n_iter1}")

# 結果を保存
filepath_case1 = save_results_2agents(
    points1, weights1, points2, weights2,
    u1_sol, u2_sol, p1_sol, p2_sol,
    obj_val1, status1,
    grid_sizes1=(NX1, NY1),
    grid_sizes2=(NX2, NY2),
    n_iter=n_iter1,
    filename="results_kandori_case1.npz"
)


In [ ]:
# 保存されたデータを読み込んで可視化
data1 = load_results_2agents("data/results_kandori_case1.npz")
print("Loaded data keys:", list(data1.keys()))
print("Agent 1 utility shape:", data1['u1_sol'].shape)
print("Agent 2 utility shape:", data1['u2_sol'].shape)
print("Agent 1 allocation shape:", data1['p1_sol'].shape)
print("Agent 2 allocation shape:", data1['p2_sol'].shape)

# データを取得
points1_arr = data1['points1']
points2_arr = data1['points2']
u1_sol = data1['u1_sol']
u2_sol = data1['u2_sol']
p1_sol = data1['p1_sol']
p2_sol = data1['p2_sol']
NX1, NY1 = data1['grid_sizes1']
NX2, NY2 = data1['grid_sizes2']


In [ ]:
# 結果の可視化: エージェント1の配分確率（財a, 財b）
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# エージェント1が型j1で、エージェント2が型j2=0（最初の型）のときの配分確率
j2_example = 0
p1_a_slice = p1_sol[0, :, j2_example]  # 財aの配分確率
p1_b_slice = p1_sol[1, :, j2_example]  # 財bの配分確率

# エージェント1の型空間を2Dグリッドに再構成
p1_a_grid = p1_a_slice.reshape(NX1, NY1)
p1_b_grid = p1_b_slice.reshape(NX1, NY1)

# 財aの配分確率
im1 = axes[0].imshow(p1_a_grid, origin='lower', aspect='auto', cmap='viridis')
axes[0].set_title(f'Agent 1: Allocation Probability for Good a\n(when Agent 2 has type {j2_example})')
axes[0].set_xlabel('Type dimension 2 (Good b)')
axes[0].set_ylabel('Type dimension 1 (Good a)')
plt.colorbar(im1, ax=axes[0])

# 財bの配分確率
im2 = axes[1].imshow(p1_b_grid, origin='lower', aspect='auto', cmap='viridis')
axes[1].set_title(f'Agent 1: Allocation Probability for Good b\n(when Agent 2 has type {j2_example})')
axes[1].set_xlabel('Type dimension 2 (Good b)')
axes[1].set_ylabel('Type dimension 1 (Good a)')
plt.colorbar(im2, ax=axes[1])

plt.tight_layout()
plt.show()


In [ ]:
# 結果の可視化: エージェント2の配分確率（財a, 財b）
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# エージェント1が型j1=0（最初の型）で、エージェント2が型j2のときの配分確率
j1_example = 0
p2_a_slice = p2_sol[0, j1_example, :]  # 財aの配分確率
p2_b_slice = p2_sol[1, j1_example, :]  # 財bの配分確率

# エージェント2の型空間を2Dグリッドに再構成
p2_a_grid = p2_a_slice.reshape(NX2, NY2)
p2_b_grid = p2_b_slice.reshape(NX2, NY2)

# 財aの配分確率
im1 = axes[0].imshow(p2_a_grid, origin='lower', aspect='auto', cmap='viridis')
axes[0].set_title(f'Agent 2: Allocation Probability for Good a\n(when Agent 1 has type {j1_example})')
axes[0].set_xlabel('Type dimension 2 (Good b)')
axes[0].set_ylabel('Type dimension 1 (Good a)')
plt.colorbar(im1, ax=axes[0])

# 財bの配分確率
im2 = axes[1].imshow(p2_b_grid, origin='lower', aspect='auto', cmap='viridis')
axes[1].set_title(f'Agent 2: Allocation Probability for Good b\n(when Agent 1 has type {j1_example})')
axes[1].set_xlabel('Type dimension 2 (Good b)')
axes[1].set_ylabel('Type dimension 1 (Good a)')
plt.colorbar(im2, ax=axes[1])

plt.tight_layout()
plt.show()


# ケース2: 異なる分布パラメータ


In [ ]:
# ケース2: エージェント1はBeta(2,2)、エージェント2はBeta(1,1)
NX1_2, NY1_2 = 8, 8
NX2_2, NY2_2 = 8, 8
BETA_PARAMS_1_2 = [
    (2.0, 2.0),  # 財a
    (2.0, 2.0),  # 財b
]
BETA_PARAMS_2_2 = [
    (1.0, 1.0),  # 財a
    (1.0, 1.0),  # 財b
]

points1_2, weights1_2 = make_tensor_grid_2d(NX1_2, NY1_2, BETA_PARAMS_1_2)
points2_2, weights2_2 = make_tensor_grid_2d(NX2_2, NY2_2, BETA_PARAMS_2_2)
print(f"#types agent1 = {len(points1_2)}")
print(f"#types agent2 = {len(points2_2)}")
print(f"Total combinations = {len(points1_2) * len(points2_2)}")


In [ ]:
status2, obj_val2, u1_sol_2, u2_sol_2, p1_sol_2, p2_sol_2, n_iter2 = solve_mechanism_2agents_iterative(
    points1_2, weights1_2, (NX1_2, NY1_2),
    points2_2, weights2_2, (NX2_2, NY2_2),
    solver=SOLVER
)

print("LP status:", status2)
print("Optimal revenue:", obj_val2)
print(f"Number of iterations: {n_iter2}")

# 結果を保存
filepath_case2 = save_results_2agents(
    points1_2, weights1_2, points2_2, weights2_2,
    u1_sol_2, u2_sol_2, p1_sol_2, p2_sol_2,
    obj_val2, status2,
    grid_sizes1=(NX1_2, NY1_2),
    grid_sizes2=(NX2_2, NY2_2),
    n_iter=n_iter2,
    filename="results_kandori_case2.npz"
)


In [ ]:
# 保存されたデータを読み込んで可視化
data2 = load_results_2agents("data/results_kandori_case2.npz")
print("Loaded data keys:", list(data2.keys()))

# データを取得
points1_arr_2 = data2['points1']
points2_arr_2 = data2['points2']
u1_sol_2 = data2['u1_sol']
u2_sol_2 = data2['u2_sol']
p1_sol_2 = data2['p1_sol']
p2_sol_2 = data2['p2_sol']
NX1_2, NY1_2 = data2['grid_sizes1']
NX2_2, NY2_2 = data2['grid_sizes2']


In [ ]:
# ケース2の結果の可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

j2_example = 0
p1_a_slice_2 = p1_sol_2[0, :, j2_example]
p1_a_grid_2 = p1_a_slice_2.reshape(NX1_2, NY1_2)

j1_example = 0
p2_a_slice_2 = p2_sol_2[0, j1_example, :]
p2_a_grid_2 = p2_a_slice_2.reshape(NX2_2, NY2_2)

im1 = axes[0].imshow(p1_a_grid_2, origin='lower', aspect='auto', cmap='viridis')
axes[0].set_title(f'Agent 1: Allocation Probability for Good a\n(Beta(2,2) × Beta(2,2))')
axes[0].set_xlabel('Type dimension 2 (Good b)')
axes[0].set_ylabel('Type dimension 1 (Good a)')
plt.colorbar(im1, ax=axes[0])

im2 = axes[1].imshow(p2_a_grid_2, origin='lower', aspect='auto', cmap='viridis')
axes[1].set_title(f'Agent 2: Allocation Probability for Good a\n(Beta(1,1) × Beta(1,1))')
axes[1].set_xlabel('Type dimension 2 (Good b)')
axes[1].set_ylabel('Type dimension 1 (Good a)')
plt.colorbar(im2, ax=axes[1])

plt.tight_layout()
plt.show()


# 結果の比較と分析


In [ ]:
# ケース1とケース2の比較（保存されたデータから読み込み）
data1 = load_results_2agents("data/results_kandori_case1.npz")
data2 = load_results_2agents("data/results_kandori_case2.npz")

print("=== ケース1 ===")
print(f"Optimal revenue: {data1['obj_val']:.6f}")
if 'n_iter' in data1:
    print(f"Number of iterations: {data1['n_iter']}")
print(f"Grid size: Agent1 {data1['grid_sizes1']}, Agent2 {data1['grid_sizes2']}")

print("\n=== ケース2 ===")
print(f"Optimal revenue: {data2['obj_val']:.6f}")
if 'n_iter' in data2:
    print(f"Number of iterations: {data2['n_iter']}")
print(f"Grid size: Agent1 {data2['grid_sizes1']}, Agent2 {data2['grid_sizes2']}")

# 効用の統計
print("\n=== エージェント1の効用統計（ケース1） ===")
print(f"Mean: {np.mean(data1['u1_sol']):.6f}")
print(f"Max: {np.max(data1['u1_sol']):.6f}")
print(f"Min: {np.min(data1['u1_sol']):.6f}")

print("\n=== エージェント2の効用統計（ケース1） ===")
print(f"Mean: {np.mean(data1['u2_sol']):.6f}")
print(f"Max: {np.max(data1['u2_sol']):.6f}")
print(f"Min: {np.min(data1['u2_sol']):.6f}")

print("\n=== エージェント1の効用統計（ケース2） ===")
print(f"Mean: {np.mean(data2['u1_sol']):.6f}")
print(f"Max: {np.max(data2['u1_sol']):.6f}")
print(f"Min: {np.min(data2['u1_sol']):.6f}")

print("\n=== エージェント2の効用統計（ケース2） ===")
print(f"Mean: {np.mean(data2['u2_sol']):.6f}")
print(f"Max: {np.max(data2['u2_sol']):.6f}")
print(f"Min: {np.min(data2['u2_sol']):.6f}")


In [ ]:
# エージェント1の配分確率の比較（エージェント2の型を固定）
data1 = load_results_2agents("data/results_kandori_case1.npz")
data2 = load_results_2agents("data/results_kandori_case2.npz")

fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# ケース1: エージェント2が型j2=0のとき
j2_example = 0
p1_a_case1 = data1['p1_sol'][0, :, j2_example].reshape(data1['grid_sizes1'])
p1_b_case1 = data1['p1_sol'][1, :, j2_example].reshape(data1['grid_sizes1'])

# ケース2: エージェント2が型j2=0のとき
p1_a_case2 = data2['p1_sol'][0, :, j2_example].reshape(data2['grid_sizes1'])
p1_b_case2 = data2['p1_sol'][1, :, j2_example].reshape(data2['grid_sizes1'])

# ケース1の可視化
im1 = axes[0, 0].imshow(p1_a_case1, origin='lower', aspect='auto', cmap='viridis')
axes[0, 0].set_title('Case 1: Agent 1 Good a\n(Beta(1,1) × Beta(1,1))')
axes[0, 0].set_xlabel('Type dimension 2 (Good b)')
axes[0, 0].set_ylabel('Type dimension 1 (Good a)')
plt.colorbar(im1, ax=axes[0, 0])

im2 = axes[0, 1].imshow(p1_b_case1, origin='lower', aspect='auto', cmap='viridis')
axes[0, 1].set_title('Case 1: Agent 1 Good b\n(Beta(1,1) × Beta(1,1))')
axes[0, 1].set_xlabel('Type dimension 2 (Good b)')
axes[0, 1].set_ylabel('Type dimension 1 (Good a)')
plt.colorbar(im2, ax=axes[0, 1])

# ケース2の可視化
im3 = axes[1, 0].imshow(p1_a_case2, origin='lower', aspect='auto', cmap='viridis')
axes[1, 0].set_title('Case 2: Agent 1 Good a\n(Beta(2,2) × Beta(2,2))')
axes[1, 0].set_xlabel('Type dimension 2 (Good b)')
axes[1, 0].set_ylabel('Type dimension 1 (Good a)')
plt.colorbar(im3, ax=axes[1, 0])

im4 = axes[1, 1].imshow(p1_b_case2, origin='lower', aspect='auto', cmap='viridis')
axes[1, 1].set_title('Case 2: Agent 1 Good b\n(Beta(2,2) × Beta(2,2))')
axes[1, 1].set_xlabel('Type dimension 2 (Good b)')
axes[1, 1].set_ylabel('Type dimension 1 (Good a)')
plt.colorbar(im4, ax=axes[1, 1])

plt.tight_layout()
plt.show()
